In [ ]:
%matplotlib inline

import sys
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import lasagne
import theano
import theano.tensor as T
from sklearn.model_selection import train_test_split

# Um Tutorial Rápido sobre *Deep Learning* e Lasagne
Juliano Henrique Foleiss

## O que é *deep learning*?

*Deep learning* (aprendizagem profunda)é um paradigma de aprendizagem de máquina onde as características são aprendidas a partir de representações alternativas dos dados de entrada. Usualmente, estas características são arranjadas de forma hierárquica, de forma que as características mais internas representam agregações e combinações de características mais externas. Isto contrasta com a abordagem tradicional *shallow learning* (aprendizagem rasa), onde o as características são determinadas com base em conhecimento especialista no domínio do problema. Neste caso, o aprendizado de máquina serve como mecanismo de separação ou combinação destas características para classificação ou regressão, sem a necessidade de derivar novas características.

Os modelos de *deep learning* consistem em múltiplas camadas de operações neurais, onde a saída de uma camada é entrada para a próxima. Usualmente, estas redes podem ser treinadas usando *backpropagation*, desde que as operações realizadas em cada camada sejam diferenciáveis. Também existem redes profundas com camadas recorrentes. Nestes casos, BTT (backpropagation through time) pode ser utilizado como procedimento de treino.

## O que é *lasagne*?

*Lasagne* é uma biblioteca escrita em *Python* que permite a prototipação rápida de redes neurais profundas. Por ser implementada sobre uma biblioteca de computação tensorial com capacidade de geração de código pra GPUs (*theano*), Lasagne pode ser usada para treinar modelos suficientemente grandes com eficiência. Mesmo sem GPU é possível desenvolver código que rode em máquinas equipadas com GPU, uma vez que o código que roda em ambas situações é exatamente o mesmo.

## Que *software* vou precisar neste tutorial?

  * Python 2.7
  * Compilador C
  * BLAS (biblioteca C/Fortran) -- **libopenblas-dev** (ubuntu / debian)
  * numpy
  * scipy
  * matplotlib
  * sklearn
  * lasagne (http://lasagne.readthedocs.io/en/latest/user/installation.html)
  * theano
  
## Dicas de instalação

  * Se você usa Linux, o Python 2.7 já está instalado no seu computador. Com certeza.
  * O compilador C mais comum é o GCC. O pacote no ubuntu é **gcc**.
  * Bibliotecas numpy e scipy são bibliotecas do Python. Recomendo usar a ferramenta pip para instalar estes pacotes, uma vez que são versões mais atualizadas que do seu gerenciador de pacotes. Basta usar
  
```
[sudo] pip install [--user] numpy scipy matplotlib sklearn
```
  Use **sudo** pra instalar para todos os usuários. Caso for instalar só para o seu usuário, use **-user**.
  * Use o link (http://lasagne.readthedocs.io/en/latest/user/installation.html) para instalar o lasagne. Este link contém instruções como instalar o theano também. Use as instruções da Seção (**Bleeding-edge version**)
  
## Sobre os exemplos de código abaixo

Os exemplos de código abaixo são adaptações do tutorial do lasagne, disponível em http://lasagne.readthedocs.io/en/latest/user/tutorial.html

Todas as explicações antes dos trechos de código e nos comentários são puramente intuições. Existem explicações muito bem elaboradas e matematicamente completas para os conceitos. No entanto, com o intuito de manter a discussão acessível e com caráter prático, resolvi omiti-las. Encorajo fortemente aos colegas que estudem estes conceitos de forma completa antes de usar as ferramentas explicadas neste tutorial


# Problema: Reconhecimento de Dígitos Manuscritos

O dataset MNIST (LeCun et. al., 1998) consiste em 60000 dígitos manuscritos centralizados em imagens monocromáticas 28x28 para treino e 10000 exemplos para teste. As classes consistem nos 10 dígitos do sistema decimal (0-9). A base está hospedada no site do Yann LeCun, e o código abaixo pode ser usado para obtê-la facilmente a partir da internet.

In [ ]:
# ################## Download and prepare the MNIST dataset ##################
# This is just some way of getting the MNIST dataset from an online location
# and loading it into numpy arrays. It doesn't involve Lasagne at all.

def load_dataset():
    # We first define a download function, supporting both Python 2 and 3.
    if sys.version_info[0] == 2:
        from urllib import urlretrieve
    else:
        from urllib.request import urlretrieve

    def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
        print("Downloading %s" % filename)
        urlretrieve(source + filename, filename)

    # We then define functions for loading MNIST images and labels.
    # For convenience, they also download the requested files if needed.
    import gzip

    def load_mnist_images(filename):
        if not os.path.exists(filename):
            download(filename)
        # Read the inputs in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
        # The inputs are vectors now, we reshape them to monochrome 2D images,
        # following the shape convention: (examples, channels, rows, columns)
        data = data.reshape(-1, 1, 28, 28)
        # The inputs come as bytes, we convert them to float32 in range [0,1].
        # (Actually to range [0, 255/256], for compatibility to the version
        # provided at http://deeplearning.net/data/mnist/mnist.pkl.gz.)
        return data / np.float32(256)

    def load_mnist_labels(filename):
        if not os.path.exists(filename):
            download(filename)
        # Read the labels in Yann LeCun's binary format.
        with gzip.open(filename, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=8)
        # The labels are vectors of integers now, that's exactly what we want.
        return data

    # We can now download and read the training and test set images and labels.
    X_train = load_mnist_images('train-images-idx3-ubyte.gz')
    y_train = load_mnist_labels('train-labels-idx1-ubyte.gz')
    X_test = load_mnist_images('t10k-images-idx3-ubyte.gz')
    y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    return X_train, y_train, X_test, y_test



A rotina abaixo mostra como montar uma rede neural usando lasagne. Note que o nome da biblioteca é bem sugestivo: a idéia é empilhar camadas de redes neurais como empilhamos deliciosas camadas de massa, queijo, molhos e outras delícias para fazer nosso prato favorito.

Note também que a função abaixo apenas define a arquitetura da rede. Esta função não define o algoritmo de otimização que vamos usar no treino, nem define uma interface de como iremos invocar as operações desta rede.

In [ ]:
def montar_mlp(var_entradas=None):
    # Esta função monta uma MLP com duas camadas ocultas com 800 neuronios cada, 
    # seguida por uma camada softmax com 10 neuronios. Na entrada é aplicado dropout de 20%
    # e nas camadas ocultas é aplicado dropout de 50%. (https://arxiv.org/pdf/1207.0580.pdf pra entender dropout)

    l_ent = lasagne.layers.InputLayer(shape=(None, 1, 28, 28),
                                      input_var=var_entradas)
    
    l_ent_drop = lasagne.layers.DropoutLayer(l_ent, p=0.2)
    
    l_oc1 = lasagne.layers.DenseLayer(l_ent_drop,
                                      num_units=800, 
                                      nonlinearity=lasagne.nonlinearities.rectify, 
                                      W=lasagne.init.GlorotUniform())

    l_oc1_drop = lasagne.layers.DropoutLayer(l_oc1, p = 0.5)
    
    l_oc2 = lasagne.layers.DenseLayer(l_oc1_drop,
                                      num_units=800, 
                                      nonlinearity=lasagne.nonlinearities.rectify, 
                                      W=lasagne.init.GlorotUniform())

    l_oc2_drop = lasagne.layers.DropoutLayer(l_oc1, p = 0.5)
    
    l_saida = lasagne.layers.DenseLayer(l_oc2_drop, 
                                      num_units=10, 
                                      nonlinearity=lasagne.nonlinearities.softmax)
    
    return l_saida
    

Com deep learning é comum que as bases de dados sejam grandes. Isto vem do fato que, como as features são aprendidas a partir dos dados, precisamos de muitos exemplos para que não haja *overfitting* apenas nos casos mais comuns. Além do mais, como a quantidade de parâmetros do modelo costuma ser muito grande, muitos exemplos são necessários para representar o espaço de busca de forma satisfatória (https://en.wikipedia.org/wiki/Curse_of_dimensionality).

Desta forma, usualmente não é possível apresentar todos os exemplos de uma só vez para a rede. Também não é eficiente apresentar apenas um por vez. Uma estratégia comum é treinar a rede em mini-batches. A função **iterate_minibatches** abaixo é uma corotina (*python generator function*) que retorna subconjuntos de tamanho *batchsize* correspondentes na primeira dimensão em inputs (X_*) e targets (Y_*).

In [ ]:
# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.
# Notice that this function returns only mini-batches of size `batchsize`.
# If the size of the data is not a multiple of `batchsize`, it will not
# return the last (remaining) mini-batch.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
N_EPOCAS = 30
TAM_BATCH = 500

print ("Carregando dataset...")
#Carregar o dataset MNIST
#Os tensores de exemplos X_* possuem as dimensões (N_EXEMPLOS, CANAIS_DE_COR, LINHAS, COLUNAS).
X_treino, Y_treino, X_teste, Y_teste = load_dataset()

#Dividir o conjunto de treino em treino e validação.
X_treino, X_val, Y_treino, Y_val = train_test_split(X_treino, Y_treino, test_size=0.2)

#Declarar as variáveis Theano para as entradas e saídas
var_entradas = T.tensor4()
var_saidas = T.ivector()

print ("Montando rede...")
rede = montar_mlp(var_entradas=var_entradas)

#Para realizar o treinamento da rede, é necessário computar uma função de custo. Neste caso,
#vamos usar a média função de entropia cruzada entre as predições da rede e os valores corretos.

#get_output retorna uma expressão theano que representa a operação "forward" da rede.
#Veja que ainda não estamos fazendo as predições! Estamos apenas montando a expressão de custo!
predicao = lasagne.layers.get_output(rede)
custo = lasagne.objectives.categorical_crossentropy(predicao, var_saidas)
custo = custo.mean()

#Com a função de custo em mãos, é possível definir a estratégia de atualização dos pesos da rede durante o treino.
#Abaixo usamos gradiente estocástico descendente com momento Nesterov.

#get_all_params é usada para pegar uma lista de todas as variáveis theano do modelo. trainable=True
#indica que queremos parametros apenas das camadas com parametros treináveis (Dropout não tem, por exemplo).
params = lasagne.layers.get_all_params(rede, trainable=True)

#neste caso, atualizacao é uma expressão theano que faz um passo e atualização nos parametros.
atualizacao = lasagne.updates.nesterov_momentum(custo, 
                                                params, 
                                                learning_rate=0.01, 
                                                momentum=0.9)

#Agora precisamos de expressões para a predição no teste, o cálculo do custo no teste e uma expressão para calculo
#da acurácia (como brinde!)

#A flag deterministic=True indica que o comportamento desta expressão deve ser determinístico.
#(Com efeito, esta flag desabilita as camadas de dropout)
teste_predicao = lasagne.layers.get_output(rede, deterministic=True)
teste_custo = lasagne.objectives.categorical_crossentropy(teste_predicao, var_saidas)
teste_custo = teste_custo.mean()

teste_ac = T.mean( T.eq( T.argmax( teste_predicao, axis=1 ), var_saidas ), dtype=theano.config.floatX )

print ("Compilando rotinas...")

#Agora sim, vamos compilar as rotinas de treino e teste que podem ser chamadas do Python! Note que tudo até agora
#é preparação para montar a rede e especificar seus parametros de treinamento.

fn_treino = theano.function([var_entradas, var_saidas], custo, updates=atualizacao)

fn_val = theano.function( [var_entradas, var_saidas], [teste_custo, teste_ac] )

custos_treino = []
custos_val = []
acs_val = []

print("Treinando modelo...")
t_ini = time.time()
for e in xrange(N_EPOCAS):
    custo_treino = 0
    bat_treino = 0
    
    for batch in iterate_minibatches(X_treino, Y_treino, TAM_BATCH):
        entradas, gabaritos = batch
        custo_treino += fn_treino(entradas, gabaritos)
        bat_treino += 1
    
    custos_treino.append( (custo_treino / bat_treino) )
    
    custo_val = 0
    ac_val = 0
    bat_val = 0
    
    for batch in iterate_minibatches(X_val, Y_val, TAM_BATCH):
        entradas, gabaritos = batch
        err, ac = fn_val(entradas, gabaritos)
        custo_val += err
        ac_val += ac
        bat_val += 1
        
    custos_val.append((custo_val / bat_val))
    acs_val.append((ac_val / bat_val))
t_fim = time.time()
print("O treino durou %.2f segundos! (média de %.2fs por época)" %  (t_fim - t_ini, (t_fim - t_ini ) / N_EPOCAS ) )
    
print("Testando modelo...")    

custo_teste = 0
ac_teste = 0
bat_teste = 0

for batch in iterate_minibatches(X_teste, Y_teste, TAM_BATCH):
    entradas, gabaritos = batch
    err, ac = fn_val(entradas, gabaritos)
    custo_teste += err
    ac_teste += ac
    bat_teste += 1

print("Erro no conjunto de TESTE: %.2f\nAcurácia no conjunto de TESTE: %.2f" % ( (custo_teste/bat_teste), (ac_teste/bat_teste)   ) )


fig, eixos = plt.subplots()
eixos.plot(custos_treino)
eixos.plot(custos_val)
eixos.set_ylabel("Erro")
eixos.set_xlabel(u"Épocas")

fig, eixos = plt.subplots()
eixos.plot(acs_val)
eixos.set_ylabel(u"Acurácia")
eixos.set_xlabel(u"Épocas")
    
